Okay, none of the labs seem to work so I'm doing much of this from scratch!

*Part 1: Webscraping Wikipedia*


In [4]:
import numpy as np
import pandas as pd
import requests

In [5]:
from bs4 import BeautifulSoup

In [6]:
# todo: remember to censor this out (for realz)
CLIENT_ID = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
CLIENT_SECRET = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
VERSION = '20200311'

In [7]:
wikipage = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
wikipage

<Response [200]>

In [8]:
wikisoup = BeautifulSoup(wikipage.text,'html.parser')

In [61]:
# relying on the fact that my data is the first table
rows = wikisoup.table.find_all('tr')

def cell_cleaner(c):
    return c.get_text().strip()

headers = list(map(cell_cleaner,rows[0].find_all('th')))

df_dict = dict()

for h in headers:
    df_dict[h]=[]
for r in rows[1:]:
    tds = r.find_all('td')
    for i in range(len(tds)):
        df_dict[headers[i]].append(cell_cleaner(tds[i]))
        
df = pd.DataFrame(df_dict)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Sweet! Step 4 complete

In [62]:
print(df.shape)

(287, 3)


*End part 1*

*Begin part 2: Loading Geospacial Data*

In [63]:
# geocoder isn't working for me so lets grab the csv

df_geo = pd.read_csv("https://cocl.us/Geospatial_data")

In [64]:
#merge into other dataframe
df = df.join(df_geo.set_index("Postal Code"),on='Postcode')
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1A,Not assigned,Not assigned,NaN,NaN
1,M2A,Not assigned,Not assigned,NaN,NaN
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636


In [65]:
# clear out the unused rows
df = df.dropna()
df = df.reset_index(drop=True)
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763


*End part 2*

*Begin part 3: Apply Clustering*

In [2]:
!pip install folium
import folium 


     |████████████████████████████████| 92kB 6.6MB/s eta 0:00:011


In [18]:
map_toronto = folium.Map(location=[df['Latitude'].mean(),df['Longitude'].mean()],zoom_start=10)
# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto


In [23]:
search_query = ''
radius = 500
LIMIT = 100
latitude = 43.75
longitude = -79.32

In [25]:
#test out foursquare api

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e6a8ed06001fe001b8e70e1'},
 'response': {'venues': [{'id': '4bc0f14c2a89ef3bdb7df188',
    'name': "Senator O'Connor Secondary School",
    'location': {'address': '60 Rowena Dr',
     'lat': 43.75078277224518,
     'lng': -79.31644920933171,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.75078277224518,
       'lng': -79.31644920933171}],
     'distance': 298,
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['60 Rowena Dr', 'Toronto ON', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d198941735',
      'name': 'College Academic Building',
      'pluralName': 'College Academic Buildings',
      'shortName': 'Academic Building',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/education/academicbuilding_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1584041678',
    'hasPerk': False},
   {'id': '4fad6522e4b08da80a56db50',


In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
toronto_venues = getNearbyVenues(names=df["Neighbourhood"],latitudes=df['Latitude'],longitudes=df['Longitude'])

Parkwoods
Victoria Village
Harbourfront
Lawrence Heights
Lawrence Manor
Queen's Park
Islington Avenue
Rouge
Malvern
Don Mills North
Woodbine Gardens
Parkview Hill
Ryerson
Garden District
Glencairn
Cloverdale
Islington
Martin Grove
Princess Gardens
West Deane Park
Highland Creek
Rouge Hill
Port Union
Flemingdon Park
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens
Eringate
Markland Wood
Old Burnhamthorpe
Guildwood
Morningside
West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
Downsview North
Wilson Heights
Thorncliffe Park
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Scarborough Village
Fairview
Henry Farm
Oriole
Northwood Park
York University
East Toronto
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
East Birchmount Park
Ionview
Kennedy Park
Bayview Village
CFB Toronto
Downsview East
The Danforth West
Riverdale
Design Exchange
Toro

In [38]:
print(toronto_venues.shape)
toronto_venues.head()

(4354, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [43]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']],prefix="",prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
toronto_onehot.head()


,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Adelaide,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,...,0.020000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000
1,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Agincourt North,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Albion Gardens,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Alderwood,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Bathurst Manor,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Bathurst Quay,0.0,0.000000,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Bayview Village,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Beaumond Heights,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Bedford Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [46]:
from sklearn.cluster import KMeans

In [47]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [49]:
len(kmeans.labels_)

204

In [55]:
toronto_grouped['Cluster'] = kmeans.labels_

In [59]:
toronto_grouped[toronto_grouped['Cluster']==1]['Neighborhood']

0                                               Adelaide
1                                              Agincourt
3                                         Albion Gardens
4                                              Alderwood
5                                         Bathurst Manor
6                                          Bathurst Quay
7                                        Bayview Village
8                                       Beaumond Heights
9                                           Bedford Park
10                                           Berczy Park
11                                           Birch Cliff
12                                     Bloordale Gardens
13                                              Brockton
14     Business Reply Mail Processing Centre 969 Eastern
15                                           CFB Toronto
16                                              CN Tower
17                                           Cabbagetown
19                 Canada Post 

In [66]:
#merge into other dataframe
df = df.join(toronto_grouped[['Neighborhood','Cluster']].set_index("Neighborhood"),on='Neighbourhood')
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,1.0
3,M6A,North York,Lawrence Heights,43.718518,-79.464763,1.0
4,M6A,North York,Lawrence Manor,43.718518,-79.464763,1.0


In [70]:
df = df.dropna()
df['Cluster'] = list(map(int,df['Cluster']))
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster
0,M3A,North York,Parkwoods,43.753259,-79.329656,0
1,M4A,North York,Victoria Village,43.725882,-79.315572,1
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,1
3,M6A,North York,Lawrence Heights,43.718518,-79.464763,1
4,M6A,North York,Lawrence Manor,43.718518,-79.464763,1


In [71]:
# okay, all that's left is to display it

map_toronto = folium.Map(location=[df['Latitude'].mean(),df['Longitude'].mean()],zoom_start=10)
colors = ['blue','red','green','orange','purple']

# add markers to map
for lat, lng, borough, neighborhood, cl in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood'],df['Cluster']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=colors[cl],
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto

And that's a wrap!!   This probably would have been way easier if the labs were working...